### NEET EET Analysis

In [1]:
!pip install imblearn

     |████████████████████████████████| 199 kB 254 kB/s eta 0:00:01
     |████████████████████████████████| 30.8 MB 5.9 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0
    Uninstalling scikit-learn-1.0:
      Successfully uninstalled scikit-learn-1.0


In [2]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

## Load and Examine Data

In [1]:
#All import go here
import pandas as pd
import numpy as np
import warnings
pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')
#Get and describe data
path = '/home/workspace/files/aryan/data_files_raw/eet_neet.csv'
df = pd.read_csv(path)
###################################33
# Fill the numeric values with nan with 0
df['TotalDisabilty'] = df['TotalDisabilty'].fillna(0)
df['TotalMentalHealth'] = df['TotalMentalHealth'].fillna(0)
df.head(2)

Results here


In [102]:
df['EET status'].unique()

array(['EET', 'NEET', nan, 'Not Known'], dtype=object)

In [2]:
## Here we will check the features which have some nan values data
## make the list of features which has missing values
#Printing the number of uique values in columns
df = df.drop(['Client Number','Prev Accomodation','A2','B2', 'E2', 'F1', 'F3','H3',
              'Ref_CN', 'Ref No', 'Service Type', 'Status',
       'Area', 'Age at Start', 'Scheme','EET status'],1)
for feature in df.columns:
    print('Feature: {}    Missing: {}    Categories: {}'.format(feature,np.round(df[feature].isnull().mean(), 4),len(df[feature].unique())))

Results here


In [3]:
df['Session per week'] = df['Num Sessions']/df['Num Weeks']
df.rename(columns = {'Do They Have Any Medical Issues':'Medical Issue',
                     'Is An Interpreter required':'Intrptr Reqd',
                     'Does the YP have a local Connection':'Local Connection'}, inplace = True)
df = df.drop(['Num Sessions','Num Weeks','Minutes per week','Economic Status','Local Connection','Intrptr Reqd','K1','A1','J1','D1','L1','E1','F2','Local Connection'],1)

df.nunique()

Reuslts here


In [96]:
#Immigration Status
df1 = df.copy(deep=True)

vari = 'B1'
df1[vari] = np.where(df1[vari]=='-','Don\'t Know / Refused', df1[vari])

vari = 'C1'
df1[vari] = np.where(df1[vari]=='-',np.NaN, df1[vari])


vari = 'G1'
df1[vari] = np.where(df1[vari]=='-',np.NaN, df1[vari])

vari = 'H1'
df1[vari] = np.where(df1[vari]=='-',np.NaN, df1[vari])

vari = 'I1'
df1[vari] = np.where(df1[vari]=='-',np.NaN, df1[vari])

vari = 'Nationality'
series = pd.value_counts(df1[vari])
mask = (series/series.sum() * 100).lt(1)
# To replace df['column'] use np.where I.e 
df1[vari] = np.where(df1[vari].isin(series[mask].index),'fill',df1[vari])

vari = 'Preferred Language'
series = pd.value_counts(df1[vari])
mask = (series/series.sum() * 100).lt(2)
# To replace df['column'] use np.where I.e 
df1[vari] = np.where(df1[vari].isin(series[mask].index),'fill',df1[vari])

vari = 'Marital Status'
df1[vari] = np.where((df1[vari]=='Couple Expecting with Children') | (df1[vari]=='Married / Civil Partnership') |
                     (df1[vari]=='Couple Expecting') |
                     (df1[vari]=='Couple with children') |(df1[vari]=='Couple'),'C', df1[vari])

vari = 'Religion'
series = pd.value_counts(df1[vari])
mask = (series/series.sum() * 100).lt(2)
# To replace df['column'] use np.where I.e 
df1[vari] = np.where(df1[vari].isin(series[mask].index),'fill',df1[vari])


one_categ_columns_with_nan = ['Medical Issue','G1','H1','I1']
df1[one_categ_columns_with_nan] = df1[one_categ_columns_with_nan].fillna('No')

In [98]:
# Imputing the vaues
df1['Time per Session'].fillna(df1['Time per Session'].mean(), inplace=True)
df1['Session per week'].fillna(df1['Session per week'].median(), inplace=True)

In [99]:
df1['Result'].value_counts()

Negative    824
Positive    413
Name: Result, dtype: int64

In [57]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pickle

######################################################################
################# Conversion to Label Encoded Data ###################
######################################################################

le = LabelEncoder()
df1['Medical Issue'] = le.fit_transform(df1['Medical Issue'])
output = open('MedicalIssue.pkl', 'wb')
pickle.dump(le, output)
output.close()

le = LabelEncoder()
df1['Initial'] = le.fit_transform(df1['Initial'])
output = open('Initial.pkl', 'wb')
pickle.dump(le, output)
output.close()

le = LabelEncoder()
df1['G1'] = le.fit_transform(df1['G1'])
output = open('G1.pkl', 'wb')
pickle.dump(le, output)
output.close()

le = LabelEncoder()
df1['H1'] = le.fit_transform(df1['H1'])
output = open('H1.pkl', 'wb')
pickle.dump(le, output)
output.close()

le = LabelEncoder()
df1['I1'] = le.fit_transform(df1['I1'])
output = open('I1.pkl', 'wb')
pickle.dump(le, output)
output.close()

# At the last, encode the result variable
le = LabelEncoder()
df1['Result'] = le.fit_transform(df1['Result'].values)
output = open('Result.pkl', 'wb')
pickle.dump(le, output)
output.close()

#Separate training features from target
X = df1.drop(['Result'], axis=1)
y = df1['Result']

#######################################
######## Train dataset = X ############
######## Test Dataset = y  ############
#######################################

######################################################################
################### Standard scaling numeric data ####################
######################################################################

ss = StandardScaler()
X[['Session per week','TotalDisabilty','TotalMentalHealth','Time per Session']] = ss.fit_transform(X[['Session per week','TotalDisabilty','TotalMentalHealth','Time per Session']])
output = open('StandardScaler.pkl', 'wb')
pickle.dump(ss, output)
output.close()


######################################################################
################### One Hot Encoding Category data ###################
######################################################################

categ = ['Gender','Preferred Language','Nationality','B1','C1','Marital Status','Sexual Orientation','Religion']
ohe = OneHotEncoder()
X_object = X[categ]
X = X.drop(categ,1)
ohe.fit(X_object)
codes = ohe.transform(X_object).toarray()
feature_names = ohe.get_feature_names(categ)
X = pd.concat([X,  pd.DataFrame(codes,columns=feature_names).astype(int)], axis=1)

output = open('OneHotEncoder.pkl', 'wb')
pickle.dump(ohe, output)
output.close()

## train test Split

In [58]:
#Now split the data into train and test Split 
seed = 999
X_Train, X_val, y_Train, y_val = train_test_split(X, y, stratify = y, test_size = 0.2, random_state=seed)

In [59]:
print("Train and Tets data are :")
print(X_Train.shape)
print(X_val.shape)
print("Teh ditribution of teh train data is :")
print(y_Train.value_counts())
print("Teh ditribution of teh test data is :")
print(y_val.value_counts())

Train and Tets data are :
(989, 50)
(248, 50)
Teh ditribution of teh train data is :
0    659
1    330
Name: Result, dtype: int64
Teh ditribution of teh test data is :
0    165
1     83
Name: Result, dtype: int64


## Augment the data

In [71]:
# Borderline SMotre with SVM
from imblearn.over_sampling import SVMSMOTE

oversample = SVMSMOTE()
X1, y1 = oversample.fit_resample(X_Train, y_Train)
y1.value_counts()

0    659
1    659
Name: Result, dtype: int64

## Grid Testing

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
outer_results_f1 = list()
for (i,(train_index, test_index)) in enumerate(cv_outer.split(X_Train,y_Train)):
    # split data
    X_train_outer, X_test_outer = X_Train.iloc[train_index], X_Train.iloc[test_index]
    y_train_outer, y_test_outer = y_Train.iloc[train_index], y_Train.iloc[test_index]
    
    # configure the cross-validation procedure
    cv_inner = StratifiedKFold(n_splits=8, shuffle=True, random_state=1)
    # define the model
    model = RandomForestClassifier(criterion='log_loss',random_state=1)
    # define search space
    space = dict()
    space['n_estimators'] = [60,80,100,120]
    #space['min_samples_split'] = [10,20,28]
    space['max_depth'] = [5,6,7]
    space['class_weight'] = ['balanced', None]
    
    # define search
    search = GridSearchCV(model, space, scoring='f1_macro', cv=cv_inner)
    # execute search
    result = search.fit(X_train_outer, y_train_outer)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test_outer)
    # evaluate the model
    acc = accuracy_score(y_test_outer, yhat)
    acc_f1=f1_score(y_test_outer, yhat, average='macro')
    # store the result
    outer_results.append(acc)
    outer_results_f1.append(acc_f1)
    
    # report progress
    print('>acc=%.3f, F1_macro=%.3f est=%.3f, cfg=%s' % (acc, acc_f1, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))
print('F1_macro: %.3f (%.3f)' % (np.mean(outer_results_f1), np.std(outer_results_f1)))

>acc=0.606, F1_macro=0.586 est=0.595, cfg={'class_weight': 'balanced', 'max_depth': 7, 'n_estimators': 120}
>acc=0.677, F1_macro=0.610 est=0.596, cfg={'class_weight': 'balanced', 'max_depth': 6, 'n_estimators': 120}
>acc=0.626, F1_macro=0.599 est=0.596, cfg={'class_weight': 'balanced', 'max_depth': 7, 'n_estimators': 120}
>acc=0.535, F1_macro=0.509 est=0.604, cfg={'class_weight': 'balanced', 'max_depth': 7, 'n_estimators': 100}
>acc=0.646, F1_macro=0.616 est=0.589, cfg={'class_weight': 'balanced', 'max_depth': 5, 'n_estimators': 120}
>acc=0.667, F1_macro=0.646 est=0.602, cfg={'class_weight': 'balanced', 'max_depth': 7, 'n_estimators': 60}
>acc=0.636, F1_macro=0.602 est=0.605, cfg={'class_weight': 'balanced', 'max_depth': 6, 'n_estimators': 60}
>acc=0.646, F1_macro=0.605 est=0.594, cfg={'class_weight': 'balanced', 'max_depth': 5, 'n_estimators': 120}
>acc=0.616, F1_macro=0.585 est=0.601, cfg={'class_weight': 'balanced', 'max_depth': 5, 'n_estimators': 120}
>acc=0.551, F1_macro=0.517 est

In [73]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

#'class_weight': {0: 0.7, 1: 0.3}, 'max_depth': 8, 'n_estimators': 60
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

best_model = RandomForestClassifier(max_depth = 6, n_estimators=60)
best_model = best_model.fit(X_Train,y_Train)
# Do the Threshold tunign and check for the possible result we can get at any threshold
yhat = best_model.predict_proba(X_val)
# keep probabilities for the positive outcome only
probs = yhat[:,1]
# define thresholds
thresholds = np.arange(0, 1, 0.001)
# evaluate each threshold
scores = [f1_score(y_val, to_labels(probs, t), average='macro') for t in thresholds]

# get best threshold
ix = np.argmax(scores)

max_score_on_val = scores[ix]
#final_model_RF = model
y_pred = np.zeros((len(X_val),1), np.uint8)
for j in range(len(X_val)):
    y_pred[j] = to_labels(probs[j], thresholds[ix])
confusion = confusion_matrix(y_val, y_pred)
print("After Threshold Tuning, results are:\n")
print('F1_macro:%.3f Threshold=%.2f' % (scores[ix],thresholds[ix]))
print("\n\n")
print("The confusion matrix for maxium score is :")
print(confusion)
print(classification_report(y_val, y_pred))

After Threshold Tuning, results are:

F1_macro:0.622 Threshold=0.43



The confusion matrix for maxium score is :
[[144  21]
 [ 53  30]]
              precision    recall  f1-score   support

           0       0.73      0.87      0.80       165
           1       0.59      0.36      0.45        83

    accuracy                           0.70       248
   macro avg       0.66      0.62      0.62       248
weighted avg       0.68      0.70      0.68       248



In [85]:
## Saving teh model to teh pickle file 
import pickle

# Thresh is 0.43
output = open('model.pkl', 'wb')
pickle.dump(best_model, output)
output.close()

## Visualize the Tree

In [20]:
!pip install graphviz

     |████████████████████████████████| 47 kB 527 kB/s eta 0:00:01


In [1]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

fig = plt.figure(figsize=(15, 10))
data = plot_tree(best_model.estimators_[1], 
          feature_names=X.columns,
          class_names=['0','1'], 
          filled=True, impurity=True, 
          rounded=True)
#fig.savefig('2224_mod_ADAsyn_RF.png', dpi=800)

Tree here !


## Explanations for the model

In [77]:
importances_unclustered = best_model.feature_importances_

In [78]:
print("Total features :")
print(len(best_model.feature_names_in_))
print('\n')
best_model.feature_names_in_

Total features :
50




array(['Initial', 'Medical Issue', 'G1', 'H1', 'I1', 'TotalDisabilty',
       'TotalMentalHealth', 'Time per Session', 'Session per week',
       'Gender_Female', 'Gender_Male', 'Gender_Trans_Gender',
       'Gender_nan', 'Preferred Language_** Believed to be English',
       'Preferred Language_** English', 'Preferred Language_fill',
       'Preferred Language_nan', 'Nationality_Non-EEA country national',
       'Nationality_Other EEA country national',
       'Nationality_UK National Resident in UK', 'Nationality_fill',
       'Nationality_nan', "B1_Don't Know / Refused", 'B1_Housing Benefit',
       'B1_No benefits', 'B1_Universal Credit', 'B1_nan',
       'C1_N/A - Not Applicable', 'C1_No', 'C1_Yes', 'C1_nan',
       'Marital Status_C', 'Marital Status_Other',
       'Marital Status_Single', 'Marital Status_Single Lone Parent',
       'Marital Status_Single Pregnant', 'Marital Status_nan',
       'Sexual Orientation_Bisexual', 'Sexual Orientation_Gay or Lesbian',
       'Sexual Ori

In [79]:
############for the Data 2224

columns = ['Initial', 'Medical Issue', 'G1', 'H1', 'I1',
       'TotalDisabilty', 'TotalMentalHealth', 'Time per Session',
       'Session per week', 'Gender','Preferred Language', 'Nationality', 'B1',
       'C1','Marital Status','Sexual Orientation', 'Religion']

one_categ_columns = ['Initial','Medical Issue','Local Connection','Intrptr Reqd','A1','D1','E1','G1','F2','H1','J1','L1','I1','TotalDisabilty',
       'TotalMentalHealth', 'Time per Session','Session per week']

pointer = 0
importances = []
for cat in columns:
    if(cat in one_categ_columns):
        values = 1
    else:
        values = len(df1[cat].unique())
    importance = sum(importances_unclustered[pointer:(pointer+values)])
    importances.append(importance)
    pointer=pointer+values

In [80]:
print("Total features detected are {} with feature importance sum {}".format(pointer, sum(importances)))

Total features detected are 50 with feature importance sum 1.0


In [2]:
forest_importances = pd.Series(importances, index=columns)
forest_importances = forest_importances.sort_values()
fig, ax = plt.subplots()
forest_importances.plot.barh(ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
plot_width, plot_height = (24,18)
plt.rcParams['figure.figsize'] = (plot_width,plot_height)
plt.rcParams['font.size']=22

Explanations here!


### Calculate the confidence intervals

Value of z = 1.64 (90%)
1.96 (95%)
2.33 (98%)
2.58 (99%)

interval = z * sqrt( (accuracy * (1 - accuracy)) / n)

In [83]:
# 95% Confidence interval
acc=0.72
interval = 1.96 * np.sqrt((acc * (1-acc))/248)
print("The accuracy of the model was {} +/- {} at the 95% confidence level.".format(acc,interval))

The accuracy of the model was 0.72 +/- 0.05588245728543139 at the 95% confidence level.


In [84]:
# 98% Confidence interval
acc=0.72
interval = 2.33 * np.sqrt((acc * (1-acc))/248)
print("The accuracy of the model was {} +/- {} at the 98% confidence level.".format(acc,interval))

The accuracy of the model was 0.72 +/- 0.0664316966709465 at the 98% confidence level.


## Temporary Work

In [91]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pickle

######################################################################
################# Conversion to Label Encoded Data ###################
######################################################################

#columns = ['Initial', 'Medical Issue', 'Intrptr Reqd', 'Local Connection',
#       'A1', 'D1', 'E1', 'F2', 'G1', 'H1', 'I1', 'J1', 'L1',
#       'TotalDisabilty', 'TotalMentalHealth', 'Time per Session',
#       'Session per week', 'Gender','Preferred Language', 'Nationality', 'B1',
#       'C1','Marital Status','Sexual Orientation', 'Religion']

def transform_input(x):
    transformed = []
    
    # Label transform the data 
    
    output = open('Initial.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['Initial']])
    transformed.append(a[0])
    
    output = open('MedicalIssue.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['Medical Issue']])
    transformed.append(a[0])
    
    output = open('Intrptr Reqd.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['Intrptr Reqd']])
    transformed.append(a[0])
    
    output = open('Local Connection.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['Local Connection']])
    transformed.append(a[0])
    
    output = open('A1.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['A1']])
    transformed.append(a[0])
    
    output = open('D1.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['D1']])
    transformed.append(a[0])
    
    output = open('E1.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['E1']])
    transformed.append(a[0])
    
    output = open('F2.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['F2']])
    transformed.append(a[0])
    
    output = open('G1.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['G1']])
    transformed.append(a[0])
    
    output = open('H1.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['H1']])
    transformed.append(a[0])
    
    output = open('I1.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['I1']])
    transformed.append(a[0])
    
    output = open('J1.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['J1']])
    transformed.append(a[0])
    
    output = open('L1.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([x['L1']])
    transformed.append(a[0])
    
    # Standard Scale the data
    output = open('StandardScaler.pkl', 'rb')
    le = pickle.load(output)
    output.close()
    a = le.transform([[x['TotalDisabilty'],x['TotalMentalHealth'],x['Time per Session'],x['Session per week']]])
    transformed.append(a[0][0])
    transformed.append(a[0][1])
    transformed.append(a[0][2])
    transformed.append(a[0][3])
    transformed = np.asarray(transformed)
    
    #One hot encoding the data finally
    output = open('OneHotEncoder.pkl', 'rb')
    ohe = pickle.load(output)
    output.close()
    codes = ohe.transform([[x['Gender'],x['Preferred Language'],x['Nationality'],x['B1'],x['C1'],x['Marital Status'],x['Sexual Orientation'],x['Religion']]]).toarray()
    transformed = np.append(transformed,codes[0])
    transformed = np.reshape(transformed,(1,58))
    
    return transformed

In [93]:
# Choosing one positive and one negative case to judge effect 
testp = df1[0:].to_dict(orient='records')[0]
testn = df1[1:].to_dict(orient='records')[0]

In [94]:
transformedp = transform_input(testp)
transformedn = transform_input(testn)
# >0.4 a positive
# <0.4 a negative
print(best_model.predict_proba(transformedp))
print(best_model.predict_proba(transformedn))

[[0.56346071 0.43653929]]
[[0.6424672 0.3575328]]


In [109]:
df1['Session per week'].describe()

count    1237.000000
mean        2.604746
std         1.490144
min         1.000000
25%         2.083333
50%         2.312500
75%         2.500000
max        17.500000
Name: Session per week, dtype: float64

In [105]:
testp['Time per Session']

107.38095238095238

In [106]:
testp['Session per week']

1.5

In [108]:
print('Session per week are {}'.format(testp['Session per week']))
##
##
##
##
#Sesion per week and Time per session are interchanged
# cHECK THE EFFECT OF THEM ON BOTH THE DATASETS
for i in np.arange(0.0,10.00,0.5):
    test1 = testp.copy()
    test1['Time per Session'] = i
    transformed = transform_input(test1)
    print("Session per week {} predicted {}".format(i,best_model.predict_proba(transformed)))

Session per week are 1.5
Session per week 0.0 predicted [[0.52054455 0.47945545]]
Session per week 1.0 predicted [[0.53571172 0.46428828]]
Session per week 2.0 predicted [[0.5450664 0.4549336]]
Session per week 3.0 predicted [[0.54571965 0.45428035]]
Session per week 4.0 predicted [[0.53666394 0.46333606]]
Session per week 5.0 predicted [[0.54499728 0.45500272]]
Session per week 6.0 predicted [[0.54792035 0.45207965]]
Session per week 7.0 predicted [[0.55181636 0.44818364]]
Session per week 8.0 predicted [[0.55181636 0.44818364]]
Session per week 9.0 predicted [[0.56669732 0.43330268]]


In [47]:
for i in df1['B1'].unique():
    test1 = test.copy()
    test1['B1'] = i
    transformed = transform_input(test1)
    print(i)
    print(best_model.predict_proba(transformed))

NameError: name 'test' is not defined

In [297]:
for i in df1['C1'].unique():
    test1 = test.copy()
    test1['C1'] = i
    transformed = transform_input(test1)
    print(i)
    print(best_model.predict_proba(transformed))

N/A - Not Applicable
[[0.48238204 0.51761796]]
No
[[0.50622681 0.49377319]]
nan
[[0.49274237 0.50725763]]
Yes
[[0.54207619 0.45792381]]


In [299]:
for i in df1['Nationality'].unique():
    test1 = test.copy()
    test1['Nationality'] = i
    transformed = transform_input(test1)
    print(i)
    print(best_model.predict_proba(transformed))

UK National Resident in UK
[[0.48238204 0.51761796]]
nan
[[0.48891763 0.51108237]]
Non-EEA country national
[[0.4239236 0.5760764]]
Other EEA country national
[[0.49857234 0.50142766]]
fill
[[0.48373828 0.51626172]]
